## SOCRATES Analysis

https://celestrak.com/SOCRATES


- NORAD Catalog Number: NORAD Catalog Numbers of the two conjuncting objects.
- Name: Common names of the two conjuncting objects. Operational status of each object is shown in brackets, using the standard CelesTrak SATCAT codes.
- Days Since Epoch: The number of days from the epoch of the NORAD two-line element set or operator ephemerides (green) used to produce the prediction until the time of closest approach for the conjunction.
- Max Probability: Given fixed spherical object radii and relative distance at the time of closest approach (TCA), the projected covariance ellipse is sized and oriented to produce the maximum probability. For these reports the ratio of major-to-minor axes of the projected covariance ellipse is set using radial, in-track, and cross-track values of 100 m, 300 m, 100 m, respectively. If the combined object radius is greater than the relative distance, the maximum probability will be one; such occurrences are not addressed here.
- Dilution Threshold (km): The standard deviation that produces the maximum probability defines the threshold of dilution. A smaller or larger standard deviation will produce a smaller probability. In the case of computing true probability with a smaller standard deviation, the data is of sufficient quality to associate low probability with low risk. For a larger standard deviation, this is not the case and the resulting true probability calculation becomes "diluted." If operating in this dilution region, the recommendation is to obtain better data and reassess the encounter. If time or resources do not permit, then the maximum probability should be used in place of the true probability.
- Min Range (km): The distance between the two conjuncting objects at the time of closest approach.
- Start (UTC): Time when the two conjuncting objects come within the computation threshold from each other.
- TCA (UTC): Time of closest approach.
- Stop (UTC): Time when the two conjuncting objects move beyond the computation threshold from each other.
- Relative Velocity (km/sec): Relative velocity of the two conjuncting objects at the time of closest approach.

In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import re

path = '../../data/socrates/'
files = [ (match[0],match[1]) for f in listdir(path) if isfile(join(path, f))  if (match:=re.search('^socrates_([0-9]{14})\.csv(\.gz)?$', f))]
files

# Build single dataset
df = pd.DataFrame()
for file,date in files:
    tmp_df = pd.read_csv(path + file)
    df = pd.concat([df,tmp_df])

    
# Fix dates and timedeltas
df['extract_date'] = pd.to_datetime(df['extract_date'], format='%Y-%m-%d %H:%M:%S.%f')
df['start_time'] = pd.to_datetime(df['start_time'], format='%Y %b %d %H:%M:%S.%f')
df['tca_time'] = pd.to_datetime(df['tca_time'], format='%Y %b %d %H:%M:%S.%f')
df['stop_time'] = pd.to_datetime(df['stop_time'], format='%Y %b %d %H:%M:%S.%f')
df['sat1_days_epoch'] = pd.to_timedelta(df['sat1_days_epoch'], 'd')
df['sat2_days_epoch'] = pd.to_timedelta(df['sat2_days_epoch'], 'd')
df['sat1_last_epoch'] = df['tca_time'] - df['sat1_days_epoch']
df['sat2_last_epoch'] = df['tca_time'] - df['sat2_days_epoch']

# Add "pair" column
df['sat_pair'] = df.apply(lambda x: x['sat1_name'] + '-' + x['sat2_name'], axis=1)

df.head()

,sat1_norad,sat1_name,sat1_days_epoch,max_prob,dil_thr_km,min_rng_km,rel_velo_kms,sat2_norad,sat2_name,sat2_days_epoch,start_time,tca_time,stop_time,extract_sort,extract_date,sat1_last_epoch,sat2_last_epoch,sat_pair
0,44421,COSMOS 2535 [+],0 days 20:12:28.800000,0.047080,0.002,0.009,0.000,44424,COSMOS 2536 [+],0 days 20:12:28.800000,2020-12-09 00:00:00.000,2020-12-09 00:03:06.880,2020-12-16 00:00:00.000,MAXPROB,2020-12-09 04:20:13.634745,2020-12-08 03:50:38.080,2020-12-08 03:50:38.080,COSMOS 2535 [+]-COSMOS 2536 [+]
1,14452,METEOR 2-10 [?],3 days 22:42:14.400000,0.005832,0.038,0.055,14.803,41302,NOAA 16 DEB [-],3 days 18:43:12,2020-12-12 03:23:20.510,2020-12-12 03:23:20.848,2020-12-12 03:23:21.186,MAXPROB,2020-12-09 04:20:13.634745,2020-12-08 04:41:06.448,2020-12-08 08:40:08.848,METEOR 2-10 [?]-NOAA 16 DEB [-]
2,26113,IMAGE [P],9 days 15:36:00,0.003850,2.112,4.014,16.806,45381,STARLINK-1258 [+],7 days 09:00:00,2020-12-15 20:01:51.336,2020-12-15 20:01:51.513,2020-12-15 20:01:51.690,MAXPROB,2020-12-09 04:20:13.634745,2020-12-06 04:25:51.513,2020-12-08 11:01:51.513,IMAGE [P]-STARLINK-1258 [+]
3,14452,METEOR 2-10 [?],3 days 21:01:26.400000,0.002003,0.066,0.093,14.804,41302,NOAA 16 DEB [-],3 days 17:02:24,2020-12-12 01:41:42.891,2020-12-12 01:41:43.228,2020-12-12 01:41:43.566,MAXPROB,2020-12-09 04:20:13.634745,2020-12-08 04:40:16.828,2020-12-08 08:39:19.228,METEOR 2-10 [?]-NOAA 16 DEB [-]
4,40935,LEMUR-2-PETER [+],5 days 04:14:52.800000,0.001938,0.012,0.044,8.494,39842,SL-16 DEB [-],7 days 06:28:48,2020-12-13 13:05:04.536,2020-12-13 13:05:05.125,2020-12-13 13:05:05.713,MAXPROB,2020-12-09 04:20:13.634745,2020-12-08 08:50:12.325,2020-12-06 06:36:17.125,LEMUR-2-PETER [+]-SL-16 DEB [-]


In [3]:
df

,sat1_norad,sat1_name,sat1_days_epoch,max_prob,dil_thr_km,min_rng_km,rel_velo_kms,sat2_norad,sat2_name,sat2_days_epoch,start_time,tca_time,stop_time,extract_sort,extract_date,sat1_last_epoch,sat2_last_epoch,sat_pair
0,44421,COSMOS 2535 [+],0 days 20:12:28.800000,4.708000e-02,0.002,0.009,0.000,44424,COSMOS 2536 [+],0 days 20:12:28.800000,2020-12-09 00:00:00.000,2020-12-09 00:03:06.880,2020-12-16 00:00:00.000,MAXPROB,2020-12-09 04:20:13.634745,2020-12-08 03:50:38.080000000,2020-12-08 03:50:38.080,COSMOS 2535 [+]-COSMOS 2536 [+]
1,14452,METEOR 2-10 [?],3 days 22:42:14.400000,5.832000e-03,0.038,0.055,14.803,41302,NOAA 16 DEB [-],3 days 18:43:12,2020-12-12 03:23:20.510,2020-12-12 03:23:20.848,2020-12-12 03:23:21.186,MAXPROB,2020-12-09 04:20:13.634745,2020-12-08 04:41:06.448000000,2020-12-08 08:40:08.848,METEOR 2-10 [?]-NOAA 16 DEB [-]
2,26113,IMAGE [P],9 days 15:36:00,3.850000e-03,2.112,4.014,16.806,45381,STARLINK-1258 [+],7 days 09:00:00,2020-12-15 20:01:51.336,2020-12-15 20:01:51.513,2020-12-15 20:01:51.690,MAXPROB,2020-12-09 04:20:13.634745,2020-12-06 04:25:51.513000000,2020-12-08 11:01:51.513,IMAGE [P]-STARLINK-1258 [+]
3,14452,METEOR 2-10 [?],3 days 21:01:26.400000,2.003000e-03,0.066,0.093,14.804,41302,NOAA 16 DEB [-],3 days 17:02:24,2020-12-12 01:41:42.891,2020-12-12 01:41:43.228,2020-12-12 01:41:43.566,MAXPROB,2020-12-09 04:20:13.634745,2020-12-08 04:40:16.828000000,2020-12-08 08:39:19.228,METEOR 2-10 [?]-NOAA 16 DEB [-]
4,40935,LEMUR-2-PETER [+],5 days 04:14:52.800000,1.938000e-03,0.012,0.044,8.494,39842,SL-16 DEB [-],7 days 06:28:48,2020-12-13 13:05:04.536,2020-12-13 13:05:05.125,2020-12-13 13:05:05.713,MAXPROB,2020-12-09 04:20:13.634745,2020-12-08 08:50:12.325000000,2020-12-06 06:36:17.125,LEMUR-2-PETER [+]-SL-16 DEB [-]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,43777,LACMA ENOCH [+],0 days 05:48:28.800000,9.341000e-08,2.249,4.946,13.754,45219,STARLINK-1187 [+],0 days 01:17:45.600000,2020-12-18 03:58:54.874,2020-12-18 03:58:54.927,2020-12-18 03:58:54.981,TIMEIN,2020-12-18 12:54:29.041352,2020-12-17 22:10:26.127000000,2020-12-18 02:41:09.327,LACMA ENOCH [+]-STARLINK-1187 [+]
2996,40286,SJ-11-08 [+],0 days 02:28:19.200000,2.151000e-07,1.781,2.716,14.866,41901,SPARK 2 [+],0 days 01:37:55.200000,2020-12-18 03:59:03.192,2020-12-18 03:59:03.475,2020-12-18 03:59:03.757,TIMEIN,2020-12-18 12:54:29.041352,2020-12-18 01:30:44.275000000,2020-12-18 02:21:08.275,SJ-11-08 [+]-SPARK 2 [+]
2997,44917,STARLINK-1098 [+],0 days 16:12:00,2.223000e-07,1.142,4.100,8.580,46689,STARLINK-1788 [+],0 days 05:58:33.600000,2020-12-18 03:59:12.294,2020-12-18 03:59:12.627,2020-12-18 03:59:12.961,TIMEIN,2020-12-18 12:54:29.041352,2020-12-17 11:47:12.627000000,2020-12-17 22:00:39.027,STARLINK-1098 [+]-STARLINK-1788 [+]
2998,15055,COSMOS 1574 [-],0 days 06:51:50.399999999,4.321000e-07,1.815,3.110,14.248,12193,DELTA 1 DEB [-],0 days 09:11:31.200000,2020-12-18 03:59:13.341,2020-12-18 03:59:13.616,2020-12-18 03:59:13.891,TIMEIN,2020-12-18 12:54:29.041352,2020-12-17 21:07:23.216000001,2020-12-17 18:47:42.416,COSMOS 1574 [-]-DELTA 1 DEB [-]


In [4]:
# I want to explore this more later to see how much of these are starlink
len(df[df['sat_pair'].str.contains('STARLINK')]['sat_pair'].unique())
# Wow 4314 unique sat_pair collision proximities!

5395

In [5]:
# Let's take one of these...
pair = 'IMAGE [P]-STARLINK-1258 [+]'
df[(df['sat_pair'] == pair)]

,sat1_norad,sat1_name,sat1_days_epoch,max_prob,dil_thr_km,min_rng_km,rel_velo_kms,sat2_norad,sat2_name,sat2_days_epoch,start_time,tca_time,stop_time,extract_sort,extract_date,sat1_last_epoch,sat2_last_epoch,sat_pair
2,26113,IMAGE [P],9 days 15:36:00,0.00385,2.112,4.014,16.806,45381,STARLINK-1258 [+],7 days 09:00:00,2020-12-15 20:01:51.336,2020-12-15 20:01:51.513,2020-12-15 20:01:51.690,MAXPROB,2020-12-09 04:20:13.634745,2020-12-06 04:25:51.513,2020-12-08 11:01:51.513,IMAGE [P]-STARLINK-1258 [+]
2,26113,IMAGE [P],9 days 15:36:00,0.00385,2.112,4.014,16.806,45381,STARLINK-1258 [+],7 days 09:00:00,2020-12-15 20:01:51.336,2020-12-15 20:01:51.513,2020-12-15 20:01:51.690,MAXPROB,2020-12-09 11:35:24.352371,2020-12-06 04:25:51.513,2020-12-08 11:01:51.513,IMAGE [P]-STARLINK-1258 [+]


In [6]:
# Values to lookup in Space-Track
coll_df = df[(df['sat_pair'] == pair)][['sat_pair','sat1_norad','sat1_last_epoch', 'sat2_norad','sat2_last_epoch']]
coll_df

,sat_pair,sat1_norad,sat1_last_epoch,sat2_norad,sat2_last_epoch
2,IMAGE [P]-STARLINK-1258 [+],26113,2020-12-06 04:25:51.513,45381,2020-12-08 11:01:51.513
2,IMAGE [P]-STARLINK-1258 [+],26113,2020-12-06 04:25:51.513,45381,2020-12-08 11:01:51.513


In [7]:
import spacetrack.operators as op
from spacetrack import SpaceTrackClient
from datetime import datetime, timedelta

spacetrack_usr = 'nmill@umich.edu'
spacetrack_pwd = open('./spacetrack_pwd.key').read()[:-1]

st = SpaceTrackClient(identity=spacetrack_usr, password=spacetrack_pwd)

In [8]:
sat1_dt = coll_df['sat1_last_epoch'].to_list()[0].to_pydatetime()
sat2_dt = coll_df['sat2_last_epoch'].to_list()[0].to_pydatetime()
sat1_epoch_range = op.inclusive_range(sat1_dt - timedelta(minutes=5), sat1_dt + timedelta(minutes=5))
sat2_epoch_range = op.inclusive_range(sat2_dt - timedelta(minutes=5), sat2_dt + timedelta(minutes=5))
sat1_epoch_range, sat2_epoch_range

('2020-12-06 04:20:51.513000--2020-12-06 04:30:51.513000',
 '2020-12-08 10:56:51.513000--2020-12-08 11:06:51.513000')

In [9]:
coll_df['sat1_norad'].to_list()[0], coll_df['sat2_norad'].to_list()[0]

(26113, 45381)

In [10]:
sat1_data = st.gp_history(norad_cat_id = coll_df['sat1_norad'].to_list()[0], epoch=sat1_epoch_range, limit=1)
sat2_data = st.gp_history(norad_cat_id = coll_df['sat2_norad'].to_list()[0], epoch=sat2_epoch_range, limit=1)

In [11]:
sat1_data

[{'CCSDS_OMM_VERS': '2.0',
  'COMMENT': 'GENERATED VIA SPACE-TRACK.ORG API',
  'CREATION_DATE': '2020-12-07T04:40:31',
  'ORIGINATOR': '18 SPCS',
  'OBJECT_NAME': 'IMAGE',
  'OBJECT_ID': '2000-017A',
  'CENTER_NAME': 'EARTH',
  'REF_FRAME': 'TEME',
  'TIME_SYSTEM': 'UTC',
  'MEAN_ELEMENT_THEORY': 'SGP4',
  'EPOCH': '2020-12-06T04:26:33.203328',
  'MEAN_MOTION': '1.68644214',
  'ECCENTRICITY': '0.76835280',
  'INCLINATION': '94.9312',
  'RA_OF_ASC_NODE': '279.5164',
  'ARG_OF_PERICENTER': '161.6313',
  'MEAN_ANOMALY': '261.1321',
  'EPHEMERIS_TYPE': '0',
  'CLASSIFICATION_TYPE': 'U',
  'NORAD_CAT_ID': '26113',
  'ELEMENT_SET_NO': '999',
  'REV_AT_EPOCH': '12745',
  'BSTAR': '-0.00820980000000',
  'MEAN_MOTION_DOT': '-0.00000790',
  'MEAN_MOTION_DDOT': '0.0000000000000',
  'SEMIMAJOR_AXIS': '29814.043',
  'PERIOD': '853.868',
  'APOAPSIS': '46343.611',
  'PERIAPSIS': '528.204',
  'OBJECT_TYPE': 'PAYLOAD',
  'RCS_SIZE': 'LARGE',
  'COUNTRY_CODE': 'US',
  'LAUNCH_DATE': '2000-03-25',
  'SI

In [12]:
sat2_data

[{'CCSDS_OMM_VERS': '2.0',
  'COMMENT': 'GENERATED VIA SPACE-TRACK.ORG API',
  'CREATION_DATE': '2020-12-08T14:46:10',
  'ORIGINATOR': '18 SPCS',
  'OBJECT_NAME': 'STARLINK-1258',
  'OBJECT_ID': '2020-019X',
  'CENTER_NAME': 'EARTH',
  'REF_FRAME': 'TEME',
  'TIME_SYSTEM': 'UTC',
  'MEAN_ELEMENT_THEORY': 'SGP4',
  'EPOCH': '2020-12-08T11:02:17.234880',
  'MEAN_MOTION': '15.05572898',
  'ECCENTRICITY': '0.00015880',
  'INCLINATION': '53.0013',
  'RA_OF_ASC_NODE': '113.3934',
  'ARG_OF_PERICENTER': '95.8375',
  'MEAN_ANOMALY': '264.2794',
  'EPHEMERIS_TYPE': '0',
  'CLASSIFICATION_TYPE': 'U',
  'NORAD_CAT_ID': '45381',
  'ELEMENT_SET_NO': '999',
  'REV_AT_EPOCH': '4103',
  'BSTAR': '0.00005370800000',
  'MEAN_MOTION_DOT': '0.00000503',
  'MEAN_MOTION_DDOT': '0.0000000000000',
  'SEMIMAJOR_AXIS': '6927.884',
  'PERIOD': '95.644',
  'APOAPSIS': '550.849',
  'PERIAPSIS': '548.649',
  'OBJECT_TYPE': 'PAYLOAD',
  'RCS_SIZE': 'LARGE',
  'COUNTRY_CODE': 'US',
  'LAUNCH_DATE': '2020-03-18',
  'S